In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.integrate import quad
import seaborn as sns
from iminuit import cost, Minuit 
from numba_stats import norm
import sys

sns.set_context('talk', font_scale=1.0)
sns.set_palette('colorblind')
source_dict = {'Am-241': 59.5409, 'Cs-137': 661.657,'Co-57': 122.06065, 'Ba-133': 356.0129}

run_table = pd.read_excel('mirion_1_2023/mirion_prototype_1/run_tag_info.xlsx')
# print(run_table.loc[run_table['preamp side'] == run_table['source side']])
# for index, row in run_table.loc[run_table['preamp side']==run_table['source side']].iterrows():
#     if row['notes'] is np.nan or row['notes']=='laptop charger plugged in, but no noticeable effect':
#         # print(row['notes'])
#         source_dict[row['source']][0].append(row['tag'])

print(source_dict)
# source_dict = {'Am241': (['XCqC', '5Cmv', 'ezim', 'iXIc'], 59.5409), 'Cs137': (['QZwJ', 'Y7Ig', 'R5w6', 'e0ym'], 661.657), \
#               'Co57': (['jWpE', 'fSeA', '9AXl', 'Qzze'], 136.4736), 'Ba133': (['dB8P', 'Uws1', 'aLj4', 'IkNH'], 356.0129)}

# line_e = 661.657
# global_gamma = 0.50
# global_CoverB = 0.13
# global_linearm = 0.028
global_sigma_ratio = 0.85

import struct
#Alex Lowell, 2020

def read(fname):

    float_t = np.double
    offset = 0
    meta = {}

    #set dtype based on filename extension
    if fname.endswith('wv'):
        reclen = int(fname.split('.')[-1].replace('wv',''))
        if reclen:
            dtype = [('time',np.uint64),('amplitude',float_t),('waveform',np.int8,(reclen,))]
        else:
            dtype = [('time',np.uint64),('amplitude',float_t)]
    elif fname.endswith('cbdump'):
        dtype = float_t
    elif fname.endswith('.dat'):
        #read first four bytes to determine file format identifier
        with open(fname,'rb') as f:
            bb = f.read(4)
            fid = struct.unpack('I',bb)[0]
            # print('format identifier: %s' % hex(fid))
        if (fid == 0x7c2b9a9f) or (fid == 0xd7d92ac6):
            nb = 37 + 32 #29 bytes of format, 32 bytes of filter info
            with open(fname,'rb') as f:
                bb = f.read(nb)
                fields = struct.unpack('=IIBIddddddd',bb)
                meta.update(zip(['format_id','raw_npoints','bit_depth','shaper_npoints','timestep','vertical','offset','rcpole','crzero','hpfpole','gain'],fields))
                # print(meta)
            if fid == 0x7c2b9a9f:
                dtype = [('time',np.uint64),('amplitude',float_t)]
            else:
                dtype = [('time',np.uint64),('amplitude',float_t),('tot',np.uint16)]
            if meta['raw_npoints']:
                npint = np.int16
                dtype.append(('waveform',npint,(meta['raw_npoints'],)))
            if meta['shaper_npoints']:
                dtype.append(('shaper',np.double,((2*meta['shaper_npoints']) + 1,)))
            offset = nb
        else:
            raise IOError('unknown format id')

    else:
        raise ValueError('bad filename extension')

    return np.fromfile(fname,dtype=dtype,offset=offset),meta

def gamma_e(E, gain):
    return (0.547 - ((5.39e-5)*E))/gain

def linearm_e(E, gain):
    return (0.0312-(0.336/E)-((3.13e-6)*E))/gain

def CoverB_e(E):
    return (0.131 + (1.44/E))

def threshold(x, x0, sigma, Eth):
    # return (1+stats.norm.cdf(x+Eth, x0, sigma))
    return stats.norm.cdf(x+Eth, x0, sigma)

def shelf(x, x0, sigma):
    return (1.-norm.cdf(x, x0, sigma))

def exp_tail(x, x0, sigma, gamma):
    return (np.exp(gamma*(x-x0)))

def linear_tail(x, x0, sigma, m):
    return (1+m*(x-x0))

def gaussian(x,x0,sigma):
    return np.exp(-(x-x0)**2/(2*sigma**2))

def func_to_pdf(x, func, Emin, Emax, *args):
    return func(x, *args)/quad(func, Emin, Emax, args=args)[0]

def gauss_plus_tail(x, BoverA, x0, sigma_gauss, gamma, CoverB, linearm, sigma_ratio):
    return (gaussian(x,x0,sigma_gauss) + BoverA*exp_tail(x, x0, sigma_gauss*sigma_ratio, gamma)*shelf(x,x0, sigma_gauss*sigma_ratio) + \
            BoverA*CoverB*linear_tail(x, x0, sigma_gauss*sigma_ratio, linearm)*shelf(x,x0, sigma_gauss*sigma_ratio))

def gauss_plus_tail_pdf(x, BoverA, x0, sigma_gauss, gamma, CoverB, linearm, sigma_ratio, Emin, Emax):
    return func_to_pdf(x, gauss_plus_tail, Emin, Emax, *[BoverA, x0, sigma_gauss, gamma, CoverB, linearm, sigma_ratio])

def gauss_plus_shelf(x, BoverA, x0, sigma_gauss, sigma_ratio):
    return (gaussian(x,x0,sigma_gauss) + BoverA*shelf(x, x0, sigma_gauss*sigma_ratio))

def gauss_plus_shelf_pdf(x, BoverA, x0, sigma_gauss, sigma_ratio, Emin, Emax):
    return func_to_pdf(x, gauss_plus_shelf, Emin, Emax, *[BoverA, x0, sigma_gauss, sigma_ratio])


{'Am-241': 59.5409, 'Cs-137': 661.657, 'Co-57': 122.06065, 'Ba-133': 356.0129}


# Loop through and try fitting

In [2]:
fwhm_dict = {}
for source in source_dict:
    fwhm_dict[source] = ([],[])
for index, row in run_table.loc[run_table['preamp side']==run_table['source side']].iterrows():
    if row['notes'] is np.nan or row['notes']=='laptop charger plugged in, but no noticeable effect':
        
        source = row['source']
        side = row['preamp side']
        d = row['tag']
        line_e = source_dict[source]
        print(d)
        data, meta = read('mirion_1_2023/mirion_prototype_1/mirion_proto1_' + side.lower() + '_side/crrc6.'+d+'.dat')
        emin = line_e*28. - 700.
        emax = line_e*28. + 200.
        hist,binedges = np.histogram(data['amplitude'], bins=100, range=(emin,emax))
        x0 = binedges[np.argmax(hist)]
        if line_e < 130:
            emin = x0-300
        else:
            emin = x0-650
        emax = x0+150
        energies = data['amplitude'][(data['amplitude']>=emin)*(data['amplitude']<=emax)]
        c = cost.UnbinnedNLL(energies, gauss_plus_tail_pdf)

        ### Set up the minimization procedure. The initialized values are from Steve's simulations
        if line_e < 150:
            m = Minuit(c, BoverA=0.5, x0=x0, sigma_gauss=35., gamma=gamma_e(line_e, 28.), CoverB=0.13, linearm=0.0, sigma_ratio=0.85, Emin=emin, Emax=emax)
            m.limits["sigma_gauss", "CoverB", "BoverA"] = (0, None)
            # m.limits["BoverA"] = (0,2.0)
            m.limits["x0"] = (emin, emax)
            m.fixed["Emin", "Emax", "sigma_ratio", "gamma", "linearm"] = True
        else:
            m = Minuit(c, BoverA=0.5, x0=x0, sigma_gauss=35., gamma=gamma_e(line_e, 28.), CoverB=CoverB_e(line_e), linearm=linearm_e(line_e, 28.), sigma_ratio=0.85, Emin=emin, Emax=emax)
            m.limits["sigma_gauss", "BoverA"] = (0, None)
            # m.limits["BoverA"] = (0,2.0)
            m.limits["x0"] = (emin, emax)
            m.fixed["Emin", "Emax", "sigma_ratio", "gamma","linearm", "CoverB"] = True

        m.migrad()
        m.hesse()

        ### Print out the FWHM of the Gaussian component
        # print('Gauss FWHM = ', 2.355*m.values['sigma_gauss']*line_e/m.values['x0'], ' +/- ', 2.355*m.errors['sigma_gauss']*line_e/m.values['x0'])
        if side=='AC':
            fwhm_dict[source][0].append(2.355*m.values['sigma_gauss']*line_e/m.values['x0'])
        else:
            fwhm_dict[source][1].append(2.355*m.values['sigma_gauss']*line_e/m.values['x0'])
        
        original_stdout = sys.stdout # Save a reference to the original standard output
        with open('mirion_1_2023/linefits/' + side + '/' + source + '/crrc6.'+d + '.fitinfo.txt', 'w') as file:
            sys.stdout = file # Change the standard output to the file we created.
            print(m)
            sys.stdout = original_stdout # Reset the standard output to its original value
            
        # print(m.values['x0']/line_e)

        fig, ax = plt.subplots(constrained_layout=True)
        ax.set_yscale('log')
        hist,binedges,_ = ax.hist(data['amplitude'], histtype="step",bins=100, range=(emin,emax))
        bin_centers = np.array((binedges[:-1] + binedges[1:]) / 2)

        def amp_to_energy(x):
            return x * line_e/m.values['x0']

        def energy_to_amp(x):
            return x * m.values['x0']/line_e

        secax = ax.secondary_xaxis('top', functions=(amp_to_energy, energy_to_amp))
        secax.set_xlabel('Energy (keV)')

        popt = m.values[:-2]
        # print(popt)
        A = np.sum(hist)*(bin_centers[1]-bin_centers[0])/\
            quad(gauss_plus_tail, np.min(energies), np.max(energies), args= tuple(popt))[0]
        B = A*m.values['BoverA']
        C = B*m.values['CoverB']

        chi_square = np.sum(np.square(A*gauss_plus_tail(bin_centers, *popt) - hist)/(A*gauss_plus_tail(bin_centers, *popt)))/(len(hist)-m.nfit)

        ax.plot(bin_centers, A*gauss_plus_tail(bin_centers, *popt),color= 'red', lw=0.5)
        ax.plot(bin_centers, A*gaussian(bin_centers, m.values['x0'], m.values['sigma_gauss']),color= 'red', ls='--', lw=0.5)
        ax.plot(bin_centers, B*exp_tail(bin_centers, m.values['x0'], m.values['sigma_gauss']*m.values['sigma_ratio'], gamma=m.values['gamma'])*shelf(bin_centers, m.values['x0'], m.values['sigma_gauss']*m.values['sigma_ratio']),color= 'red', ls='--', lw=0.5)
        ax.plot(bin_centers, C*linear_tail(bin_centers, m.values['x0'], m.values['sigma_gauss']*m.values['sigma_ratio'], m.values['linearm'])*shelf(bin_centers, m.values['x0'], m.values['sigma_gauss']*m.values['sigma_ratio']),color= 'red', ls='--', lw=0.5)

        ax.set_ylim(bottom=1.)
        ax.set_xlim((emin, emax))
        ax.set_xlabel('pulse amplitude')
        ax.set_ylabel('counts')

        ax.text(0.05, 0.85, \
                'FWHM = ' + str(round(2.355*m.values['sigma_gauss']*line_e/m.values['x0'], 2))  + r'$\pm$' + str(round(2.355*m.errors['sigma_gauss']*line_e/m.values['x0'], 2)) +' keV', \
               transform = ax.transAxes)
        ax.text(0.05, 0.75, r'$\chi^2_{\nu}=$' + str(round(chi_square,2)), transform = ax.transAxes)

        # plt.show()
        plt.savefig('mirion_1_2023/linefits/' + side + '/' + source + '/crrc6.'+d + '.pdf')
        plt.close()

iXIc
ezim
QZwJ
Y7Ig
dB8P
Uws1
jWpE
fSeA
5Cmv
XCqC
R5w6
e0ym
9AXl
Qzze
aLj4
IkNH
HfjA
yN5Q
KTZk
KSpa
CXsV
5UMx
XFPA
vgsg
V4T1
Jeog
bFYc
jVfj
sRnu
Jwml
20SP
Mj9T


In [25]:
for source in source_dict:
    print(source)
    print(np.mean(fwhm_dict[source][0]),np.mean(fwhm_dict[source][1]))

Am-241
3.7333625827335997 3.315574473262613
Cs-137
4.554053321435571 3.9641447691171243
Co-57
3.9174956437044384 3.4779070647788064
Ba-133
3.927435654908038 3.4457422022802704
